In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F

import scipy
import scipy.signal
import scipy.io

import matplotlib.pyplot as plt
plt.style.use('dark_background')

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from joblib.parallel import Parallel, delayed

import shutil


from torch.utils.tensorboard import SummaryWriter



# from skorch.callbacks import LRScheduler, BatchScoring
# from skorch.helper import SliceDataset

# from braindecode.datasets import WindowsDataset, BaseConcatDataset
# from braindecode.util import set_random_seeds
# from braindecode.models import ShallowFBCSPNet, Deep4Net
# from braindecode.models.util import to_dense_prediction_model
# from braindecode.models.modules import Expression
# from braindecode import EEGRegressor

In [3]:
# # Remove all logs 

# path = os.path.abspath("logs")
# print(path)
# if os.path.exists(path):
#     shutil.rmtree(path)
# os.makedirs(path, exist_ok=True)

In [4]:
torch.cuda.empty_cache() 

## Get data + create dataloader



In [5]:
folder_path = "/vol/aimspace/users/dena/Documents/mae/data/lemon"


In [6]:
data_raw_test = torch.load(os.path.join(folder_path, "data_raw_test.pt"))
data_raw_train = torch.load(os.path.join(folder_path, "data_raw_train.pt"))
data_raw_val = torch.load(os.path.join(folder_path, "data_raw_val.pt"))

labels_raw_test = torch.load(os.path.join(folder_path, "labels_raw_test.pt"))
labels_raw_train = torch.load(os.path.join(folder_path, "labels_raw_train.pt"))
labels_raw_val = torch.load(os.path.join(folder_path, "labels_raw_val.pt"))

NOVEL ATTEMPT


In [7]:
h_params = {
    "num_epochs": 100, 
    "batch_size": 64, 
    "hidden_size1": 4096, # 512, 
    "hidden_size2": 128, 
    "lr": 0.002, 

    "number_samples": 4  # either None for all samples or int  
}

In [8]:
time_steps = 2000

def split_dataset(raw_dataset, labels_raw, time_steps): 
    num_chunks_per_sample = raw_dataset.shape[-1] // time_steps  # This calculates to 15
    
    # Split the tensor along the last dimension; discard last chunk if it does not have the full time_steps
    chunks_data = torch.split(raw_dataset, time_steps, dim=2) #, dim=-1)
    print(chunks_data[0].shape)
    stacked_tensor = torch.stack(chunks_data, dim=1)
    
    return stacked_tensor

print("Val")
data_val = split_dataset(data_raw_val, labels_raw_val, time_steps)
print("Train")
data_train = split_dataset(data_raw_train, labels_raw_train, time_steps)
print("Test")
data_test = split_dataset(data_raw_test, labels_raw_test, time_steps)

Val
torch.Size([32, 61, 2000])
Train
torch.Size([150, 61, 2000])
Test
torch.Size([32, 61, 2000])


In [9]:
class EEGDataset(Dataset):
    def __init__(self, X, y, number_samples=None):
        if number_samples: 
            indices = np.random.choice(range(len(X[0])), number_samples, replace=False)
            self.X = X[indices]
            self.y = y[indices]
        else: 
            self.X = X
            self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        participant_trials = self.X[idx]
        label = self.y[idx]
        return participant_trials, label


train_dataset = EEGDataset(data_train, labels_raw_train, h_params["number_samples"]) #, number_samples=32) #!!! CHANGE AGAIN!! 
val_dataset = EEGDataset(data_val, labels_raw_val, h_params["number_samples"]) #, number_samples=32) #!!! CHANGE AGAIN!! 
test_dataset = EEGDataset(data_test, labels_raw_test, h_params["number_samples"]) #, number_samples=32) #!!! CHANGE AGAIN!! 

In [12]:
# model = InspiredEEGNet(n_channels=15, n_classes=1, input_time_length=2000)
model = SimpleNN()
print(model)
# 1830000

SimpleNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=122000, out_features=4096, bias=True)
  (maxpool1): MaxPool2d(kernel_size=(1, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc2): Linear(in_features=4096, out_features=128, bias=True)
  (fc2_1): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
  (activation): Sigmoid()
)


In [13]:
def reset_weights(m):
    '''
    Resets model weights to avoid weight leakage across folds during cross-validation.
    Applies to layers with explicit weight parameters (e.g., Conv2d, Linear).
    '''
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.zeros_(m.bias)
    # Add other layer types here if necessary. For example:
    elif isinstance(m, nn.BatchNorm2d):
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)

In [14]:
# Create Dataloaders
train_loader = DataLoader(train_dataset, batch_size=h_params["batch_size"], shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=h_params["batch_size"], shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=h_params["batch_size"], shuffle=False, num_workers=4)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# model = SimpleNN()
criterion = nn.BCEloss() #MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=h_params["lr"]) 
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 


cuda:0


In [15]:
def perform_cross_validation(data, num_folds, hyperparameters, unique_identifier):

    rate_of_updates = 5 
    
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_results = []


    for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
        writer_name = f'logs/{unique_identifier}-fold_{fold}'
        writer = SummaryWriter(log_dir=writer_name)

        # Log hyperparameters
        h_params_text = "Hyperparameters:  "
        for key, value in hyperparameters.items():
            h_params_text += f"{key}: {value}, "
        writer.add_text(tag='EEG_shallowFBCSPNet', text_string=h_params_text)

        # writer.add_hparams(hparam_dict=hyperparameters, )

        train_subset = torch.utils.data.Subset(data, train_idx)
        val_subset = torch.utils.data.Subset(data, val_idx)

        # Create DataLoaders for the current fold
        train_loader = DataLoader(train_subset, batch_size=h_params["batch_size"], shuffle=True) #, num_workers=4)
        val_loader = DataLoader(val_subset, batch_size=h_params["batch_size"], shuffle=False) #, num_workers=4)

        # Reset model to initial state
        # model = model.to(device)
        model.apply(reset_weights)
        # (Re)initialize the optimizer if needed since model weights are reset
        optimizer = torch.optim.Adam(model.parameters(), lr=h_params["lr"])

        for epoch in range(hyperparameters["num_epochs"]): 
            model.train()
            train_running_loss = 0.0
            
            for i, (X,y) in enumerate(train_loader):
            # Ensure data is on the correct device

                X, y = X.to(device), y.to(device)
                
                optimizer.zero_grad()
                
                outputs = model(X)
                loss = criterion(outputs, y)
                
                loss.backward()
                optimizer.step()
                
                train_running_loss += np.sqrt(loss.item()) 
                writer.add_scalar('Loss_in_years/train', train_running_loss, epoch * len(train_loader) + i)

            avg_train_loss = train_running_loss/len(train_loader)

            # if epoch % rate_of_updates == 0: 
            #     print(f'Fold {fold+1}, Epoch [{epoch+1}/{h_params["num_epochs"]}], Train Loss: {avg_train_loss:.4f}')
            
            print(f'Fold {fold+1}, Epoch [{epoch+1}/{h_params["num_epochs"]}], Train Loss: {avg_train_loss:.4f}')
        
            # Validation phase
            model.eval()
            val_running_loss = 0.0
            
            with torch.no_grad():
                for i, (X,y) in enumerate(val_loader):
                    X, y = X.to(device), y.to(device)
                    
                    outputs = model(X)
                    val_loss = criterion(outputs, y)
                    
                    val_running_loss += np.sqrt(val_loss.item())

                    # avg_val_loss = val_running_loss / len(val_loader)
                    writer.add_scalar('Loss_in_years/val', np.sqrt(val_loss.item()), epoch * len(val_loader) + i)

            avg_val_loss = val_running_loss / len(val_loader)

        writer.add_hparams(hparam_dict=hyperparameters, metric_dict={"Loss_in_years/train": avg_train_loss, "Loss_in_years/val": avg_val_loss})

            
        print(f'Fold {fold+1}, Epoch [{epoch+1}/{h_params["num_epochs"]}], Validation Loss: {avg_val_loss:.4f}')
        # writer.add_scalar('Loss_in_years/val', avg_val_loss, epoch * len(val_loader) + i)

        # Log fold results
        fold_results.append(avg_val_loss)

        # writer.add_hparams(hparam_dict=hyperparameters, metric_dict={"Loss_in_years/train": train_running_loss, "Loss_in_years/val": val_running_loss})


    # Evtl hier logs für epochs? 

    # After all folds
    print(f'Individual Validation Losses for all folds: {fold_results}')
    print(f'Average Validation Loss across all folds: {np.mean(fold_results):.4f}')

In [16]:
def perform_cross_validation_on_the_simple_fcn(data, num_folds, hyperparameters, unique_identifier):

    rate_of_updates = 5 
    
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_results = []


    for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
        writer_name = f'logs/{unique_identifier}-fold_{fold}'
        writer = SummaryWriter(log_dir=writer_name)

        # Log hyperparameters
        h_params_text = "Hyperparameters:  "
        for key, value in hyperparameters.items():
            h_params_text += f"{key}: {value}, "
        writer.add_text(tag='EEG_shallowFBCSPNet', text_string=h_params_text)

        # writer.add_hparams(hparam_dict=hyperparameters, )

        train_subset = torch.utils.data.Subset(data, train_idx)
        val_subset = torch.utils.data.Subset(data, val_idx)

        # Create DataLoaders for the current fold
        train_loader = DataLoader(train_subset, batch_size=h_params["batch_size"], shuffle=True) #, num_workers=4)
        val_loader = DataLoader(val_subset, batch_size=h_params["batch_size"], shuffle=False) #, num_workers=4)

        # Reset model to initial state
        # model = model.to(device)
        model.apply(reset_weights)
        # (Re)initialize the optimizer if needed since model weights are reset
        optimizer = torch.optim.Adam(model.parameters(), lr=h_params["lr"])

        for epoch in range(hyperparameters["num_epochs"]): 
            model.train()
            train_running_loss = 0.0
            
            for i, (X_batch,y_batch) in enumerate(train_loader): 
                for X in X_batch.squeeze(0): #, y_batch): #zip(X_batch[:,i,:,:],y_batch[:,i,:,:]): 
            # Ensure data is on the correct device
                    print(X.shape)

                    X, y = X.unsqueeze(0).to(device), y_batch.to(device)
                    
                    optimizer.zero_grad()
                    
                    outputs = model(X)
                    loss = criterion(outputs, y)
                    
                    loss.backward()
                    optimizer.step()
                    
                train_running_loss += np.sqrt(loss.item()) 
                writer.add_scalar('Loss_in_years/train', train_running_loss, epoch * len(train_loader) + i)

            avg_train_loss = train_running_loss/len(train_loader)

            # if epoch % rate_of_updates == 0: 
            #     print(f'Fold {fold+1}, Epoch [{epoch+1}/{h_params["num_epochs"]}], Train Loss: {avg_train_loss:.4f}')
            
            print(f'Fold {fold+1}, Epoch [{epoch+1}/{h_params["num_epochs"]}], Train Loss: {avg_train_loss:.4f}')
        
            # Validation phase
            model.eval()
            val_running_loss = 0.0
            
            with torch.no_grad():
                for i, (X_batch,y_batch) in enumerate(val_loader):
                    
                    for X in X_batch.squeeze(0): #X, y in zip(X_batch[:,i,:,:],y_batch[:,i,:,:]): 
                        X, y = X.unsqueeze(0).to(device), y_batch.to(device)
                        # X, y = X.to(device), y.to(device)
                        
                        outputs = model(X)
                        val_loss = criterion(outputs, y)
                        
                        val_running_loss += np.sqrt(val_loss.item())

                        # avg_val_loss = val_running_loss / len(val_loader)
                        writer.add_scalar('Loss_in_years/val', np.sqrt(val_loss.item()), epoch * len(val_loader) + i)

            avg_val_loss = val_running_loss / len(val_loader)

        writer.add_hparams(hparam_dict=hyperparameters, metric_dict={"Loss_in_years/train": avg_train_loss, "Loss_in_years/val": avg_val_loss})

            
        print(f'Fold {fold+1}, Epoch [{epoch+1}/{h_params["num_epochs"]}], Validation Loss: {avg_val_loss:.4f}')
        # writer.add_scalar('Loss_in_years/val', avg_val_loss, epoch * len(val_loader) + i)

        # Log fold results
        fold_results.append(avg_val_loss)

        # writer.add_hparams(hparam_dict=hyperparameters, metric_dict={"Loss_in_years/train": train_running_loss, "Loss_in_years/val": val_running_loss})


    # Evtl hier logs für epochs? 

    # After all folds
    print(f'Individual Validation Losses for all folds: {fold_results}')
    print(f'Average Validation Loss across all folds: {np.mean(fold_results):.4f}')

In [ ]:
def perform_cross_validation2(X_data,y_data, num_folds, hyperparameters, unique_identifier):

    rate_of_updates = 5 
    
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_data)):

        # Tensorboard
        writer_name = f'logs/{unique_identifier}-fold_{fold}'
        writer = SummaryWriter(log_dir=writer_name)

        # Tensorboard: Log hyperparameters
        h_params_text = "Hyperparameters:  "
        for key, value in hyperparameters.items():
            h_params_text += f"{key}: {value}, "
        writer.add_text(tag='EEG_shallowFBCSPNet', text_string=h_params_text)

        # writer.add_hparams(hparam_dict=hyperparameters, )

        train_subset = torch.utils.data.Subset(X_data, train_idx)
        val_subset = torch.utils.data.Subset(X_data, val_idx)

        # Create DataLoaders for the current fold
        train_loader = DataLoader(train_subset, batch_size=h_params["batch_size"], shuffle=True) #, num_workers=4)
        val_loader = DataLoader(val_subset, batch_size=h_params["batch_size"], shuffle=False) #, num_workers=4)

        # Reset model to initial state
        # model = model.to(device)
        model.apply(reset_weights)
        # (Re)initialize the optimizer if needed since model weights are reset
        optimizer = torch.optim.Adam(model.parameters(), lr=h_params["lr"])

        for epoch in range(hyperparameters["num_epochs"]): 
            model.train()
            train_running_loss = 0.0
            
            for i, (X,y) in enumerate(train_loader):
            # Ensure data is on the correct device

                X, y = X.to(device), y.to(device)
                
                optimizer.zero_grad()
                
                outputs = model(X)
                loss = criterion(outputs, y)
                
                loss.backward()
                optimizer.step()
                
                train_running_loss += np.sqrt(loss.item()) 
                writer.add_scalar('Loss_in_years/train', train_running_loss, epoch * len(train_loader) + i)

            avg_train_loss = train_running_loss/len(train_loader)

            # if epoch % rate_of_updates == 0: 
            #     print(f'Fold {fold+1}, Epoch [{epoch+1}/{h_params["num_epochs"]}], Train Loss: {avg_train_loss:.4f}')
            
            print(f'Fold {fold+1}, Epoch [{epoch+1}/{h_params["num_epochs"]}], Train Loss: {avg_train_loss:.4f}')
        
            # Validation phase
            model.eval()
            val_running_loss = 0.0
            
            with torch.no_grad():
                for i, (X,y) in enumerate(val_loader):
                    X, y = X.to(device), y.to(device)
                    
                    outputs = model(X)
                    val_loss = criterion(outputs, y)
                    
                    val_running_loss += np.sqrt(val_loss.item())

                    # avg_val_loss = val_running_loss / len(val_loader)
                    writer.add_scalar('Loss_in_years/val', np.sqrt(val_loss.item()), epoch * len(val_loader) + i)

            avg_val_loss = val_running_loss / len(val_loader)

        writer.add_hparams(hparam_dict=hyperparameters, metric_dict={"Loss_in_years/train": avg_train_loss, "Loss_in_years/val": avg_val_loss})

            
        print(f'Fold {fold+1}, Epoch [{epoch+1}/{h_params["num_epochs"]}], Validation Loss: {avg_val_loss:.4f}')
        # writer.add_scalar('Loss_in_years/val', avg_val_loss, epoch * len(val_loader) + i)

        # Log fold results
        fold_results.append(avg_val_loss)

        # writer.add_hparams(hparam_dict=hyperparameters, metric_dict={"Loss_in_years/train": train_running_loss, "Loss_in_years/val": val_running_loss})


    # Evtl hier logs für epochs? 

    # After all folds
    print(f'Individual Validation Losses for all folds: {fold_results}')
    print(f'Average Validation Loss across all folds: {np.mean(fold_results):.4f}')

In [32]:
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetClassifier


NameError: name 'X' is not defined

In [48]:
print(data_train[0].shape)
print(data_train[1].shape)

print(data_raw_train[:,:,:2000].shape)
print(labels_raw_train.shape)

torch.Size([15, 61, 2000])
torch.Size([15, 61, 2000])
torch.Size([150, 61, 2000])
torch.Size([150, 1])


In [51]:
# Only one sample per participant! (Just the 2000 first time points)

from skorch import NeuralNetRegressor

X = data_raw_train[:,:,:2000]
y = labels_raw_train

net = NeuralNetRegressor(
    SimpleNN,
    max_epochs=200,
    lr=0.1,
    device=device
)

net.fit(X,y)
y_proba = net.predict_proba(X)

start
torch.Size([120, 61, 2000])
torch.Size([120, 122000])
start
torch.Size([30, 61, 2000])
torch.Size([30, 122000])
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1     1982.4697    52382.6602  0.0930
start
torch.Size([120, 61, 2000])
torch.Size([120, 122000])
start
torch.Size([30, 61, 2000])
torch.Size([30, 122000])
      2    50935.3867     5688.5630  0.0859
start
torch.Size([120, 61, 2000])
torch.Size([120, 122000])
start
torch.Size([30, 61, 2000])
torch.Size([30, 122000])
      3     6248.6836     3691.7283  0.0867
start
torch.Size([120, 61, 2000])
torch.Size([120, 122000])
start
torch.Size([30, 61, 2000])
torch.Size([30, 122000])
      4     4150.0469     2430.0427  0.0864
start
torch.Size([120, 61, 2000])
torch.Size([120, 122000])
start
torch.Size([30, 61, 2000])
torch.Size([30, 122000])
      5     2806.9199     1635.5948  0.0854
start
torch.Size([120, 61, 2000])
torch.Size([120, 122000])
start
torch.Size([30, 61, 2000])
torch.Siz

In [ ]:
# https://github.com/skorch-dev/skorch?tab=readme-ov-file
# deactivate skorch-internal train-valid split and verbose logging
model.set_params(train_split=False, verbose=0)
params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 20],
    'module__num_units': [10, 20],
}
gs = GridSearchCV(model, params, refit=False, cv=3, scoring='accuracy', verbose=2)

gs.fit(X, y)
print("best score: {:.3f}, best params: {}".format(gs.best_score_, gs.best_params_))


In [31]:
LR = [0.001]#, 0.0005, 0.002]
BATCH_SIZES = [1] #, 4, 16, 32, 64]

NUMBER_SAMPLES = [1]#, 4, 16, None]

# Next attempt: SimpleNN
for lr in LR: 
    for batch_size in BATCH_SIZES: 
        for number_samples in NUMBER_SAMPLES: 
            now = datetime.now()
            # unique_identifier = f"EEG_shallowFBCSPNet_-{now.strftime('%Y_%m_%d-%H_%M')}" #("%Y%m%d_%H%M%S")
            unique_identifier = f"Simple_NN-{now.strftime('%Y_%m_%d-%H_%M')}" #("%Y%m%d_%H%M%S")

            model = model.to(device)
            print("datasets")
            print(train_dataset[0][1].shape)

            # full_train_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])
            # print(full_train_dataset[0][0].shape) #x 
            # I AM UNSURE IF THIS WORKS - so I replaced it for now with only the training set. 
            h_params["lr"] = lr
            h_params["batch_size"] = batch_size
            h_params["number_samples"] = number_samples

            perform_cross_validation2(train_dataset[0], 5, h_params, unique_identifier)
            # perform_cross_validation_on_the_simple_fcn(full_train_dataset, 5, h_params, unique_identifier)

datasets
torch.Size([1])


NameError: name 'perform_cross_validation2' is not defined

In [17]:
## WORKS
now = datetime.now()
# unique_identifier = f"EEG_shallowFBCSPNet_-{now.strftime("%Y_%m_%d-%H_%M")}" #("%Y%m%d_%H%M%S")

# model = model.to(device)

# full_train_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])

# perform_cross_validation(full_train_dataset, 5, h_params, unique_identifier)

In [65]:
# # Unique Identifier to encode day and time
# now = datetime.now()
# unique_identifier = now.strftime("%Y_%m_%d-%H_%M") #("%Y%m%d_%H%M%S")

# writer_name = f'logs/EEG_shallowFBCSPNet_Adam-{unique_identifier}'
# writer = SummaryWriter(writer_name)
# print(writer_name)

# writer.add_hparams(h_params, {})


# # Initializing the list for storing the loss 
# train_loss_history = [] # loss

# model = model.to(device)

# for epoch in range(h_params["num_epochs"]):
#     model.train() 
#     running_loss = 0.0

#     for i, (X, y) in enumerate(train_loader): 

#         X, y = X.to(device), y.to(device)
#         y = y.unsqueeze(-1)  

#         optimizer.zero_grad() 

#         outputs = model(X) 
#         loss = criterion(outputs, y)

#         loss.backward()  
#         optimizer.step()  

#         running_loss += loss.item()
#         loss_in_years = torch.sqrt(loss)
#         writer.add_scalar('Loss_in_years/train', loss_in_years.item(), epoch * len(val_loader) + i)
#         # writer.add_hparams(h_params, {'hparam/loss_train_in_y':loss_in_years.item()}) #, 'hparam/loss_val_in_y': np.sqrt(avg_val_loss)})

#         # Print statistics to console
#         frequency_of_updates = 4
#         if i % frequency_of_updates == frequency_of_updates - 1:
#             current_loss = np.sqrt(running_loss / frequency_of_updates)
#             print(f"[Epoch {epoch+1}, Iteration {i+1}] loss: {current_loss:.3f}")
#             train_loss_history.append(current_loss)
#             running_loss = 0.0



#     # Validation phase
#     model.eval()
#     val_running_loss = 0.0
#     val_total = 0

#     with torch.no_grad():
#         for i, (X, y) in enumerate(val_loader):
#             X, y = X.to(device), y.to(device)
#             y = y.unsqueeze(1)  # Ensure y is the correct shape for comparison

#             outputs = model(X)
#             val_loss = criterion(outputs, y)

#             val_running_loss += val_loss.item()
#             val_total += y.size(0)

#     avg_val_loss = val_running_loss / len(val_loader)
#     writer.add_scalar('Loss/val', np.sqrt(avg_val_loss), epoch)
#     # writer.add_hparams(h_params, {'hparam/loss_val_in_y': np.sqrt(avg_val_loss)})


#     print(f'Epoch [{epoch+1}/{h_params["num_epochs"]}], Validation Loss: {np.sqrt(avg_val_loss):.4f}')


logs/EEG_shallowFBCSPNet_Adam-2024_03_08-01_28


Epoch [1/50], Validation Loss: 23.7685
Epoch [2/50], Validation Loss: 23.7464
Epoch [3/50], Validation Loss: 23.7250
Epoch [4/50], Validation Loss: 23.7179
Epoch [5/50], Validation Loss: 23.7137
Epoch [6/50], Validation Loss: 23.7082
Epoch [7/50], Validation Loss: 23.6957
Epoch [8/50], Validation Loss: 23.6604
Epoch [9/50], Validation Loss: 23.6078
Epoch [10/50], Validation Loss: 23.5330
Epoch [11/50], Validation Loss: 23.4387
Epoch [12/50], Validation Loss: 23.3394
Epoch [13/50], Validation Loss: 23.2524
Epoch [14/50], Validation Loss: 23.1239
Epoch [15/50], Validation Loss: 22.9867
Epoch [16/50], Validation Loss: 22.7739
Epoch [17/50], Validation Loss: 22.4569
Epoch [18/50], Validation Loss: 21.8786
Epoch [19/50], Validation Loss: 20.9543
Epoch [20/50], Validation Loss: 19.7718
Epoch [21/50], Validation Loss: 18.4017
Epoch [22/50], Validation Loss: 17.2036
Epoch [23/50], Validation Loss: 16.6899
Epoch [24/50], Validation Loss: 16.0767
Epoch [25/50], Validation Loss: 15.4445
Epoch [26

Good Graphs of Samples: 

In [ ]:
# # Plot of one person and one sample over all 
# ages = []

# # Get one batch 
# for j, data_train in enumerate(train_loader, 0): 
#     print(j)
#     print(f"train_loader sample {j} is {len(data_train)}, {len(data_train[0])}, {len(data_train[0][0])}, {len(data_train[0][0][0])} big.")
#     # x,y; people, channels; timesteps


#     # Iterate through all people of one batch 
#     for data_of_one_sample, age in zip(data_train[0], data_train[1]): # x of people, y of people
#         plt.plot(data_of_one_sample.T)
#         ages.append(age)
#         plt.title(f"Data of one sample of a {age[0]} year old participant")
#         plt.show()
#     # plt.plot(data_train[j][0][i])

#     break

# ages



In [ ]:
# means = []
# y_values = []

# for i, data in enumerate(train_loader, 0):

#     print(data[0].shape) #[0])
#     # Compute the mean of each datapoint (assuming inputs are tensors of shape [batch_size, 61, 2000])

#     mean = data[0].mean(dim=[1, 2])  # This computes the mean across both dimensions for each item in the batch
#     means.extend(mean.tolist())  # Convert to list and store
#     y_values.extend(data[1].flatten().tolist())  # Flatten labels and store

# # Now plot the means vs. y-values
# plt.figure(figsize=(10, 6))
# plt.scatter(means, y_values)
# plt.title('Mean of Each Measurement vs. Age')
# plt.xlabel('Mean of All EEG Measurementpoints')
# plt.ylabel('Age')
# plt.show()

In [ ]:
# # Get number of parameters
# # def count_parameters(model):
# #     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# # Determine size in bytes of model 
# def model_size_in_bytes(model):
#     param_size = 0
#     for param in model.parameters():
#         param_size += param.nelement() * param.element_size()
#     size_in_bytes = param_size  # Only parameters, no gradients or activations
#     return size_in_bytes

# # model_parameters = count_parameters(model)
# # print(f"Number of trainable parameters: {model_parameters}")

# model_size_bytes = model_size_in_bytes(model)
# model_size_mibs = model_size_bytes / (1024 ** 2)  # Convert bytes to MiB

# print(f"Model size: {model_size_bytes} bytes")
# print(f"Model size: {model_size_mibs} MiB")


# # Goal: Have the GPU Memory be at least 3/4 times as large (GPU has 16384 MiB >> max 4096 to 5461). 

Model size: 2000962564 bytes
Model size: 1908.2666053771973 MiB
